### Paso 1 - Definir la estructura del esquema para "languages"

In [0]:
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
languages_schema = StructType([
    StructField("languageId", IntegerType(), True),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])

### Paso 2 - Leer el archivo languages csv

In [0]:
language = spark.read.schema(languages_schema).option("header", "true").csv(f"/mnt/historymovie1416/bronze/{v_file_date}/language.csv")

### Paso 3 - Seleccionar las columnas necesarias y renombrarlas

In [0]:
language_df = language.select(
    col("languageId").alias("language_id"),
    col("languageName").alias("language_name"),
    current_timestamp().alias("ingesion_date")
)
    

### Paso 4 - Agregar la columna de current_timestamp 

In [0]:
language_final = language_df.withColumn("environment", lit(v_environment)).withColumn("file_date", lit(v_file_date))

### Paso 5 - Escribir en el contenedor silver el dataFrame languages

In [0]:
# language_final.write.mode("overwrite").parquet("/mnt/historymovie1416/silver/language")
language_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language")

In [0]:
%sql
SELECT * FROM movie_silver.language;

In [0]:
dbutils.notebook.exit("Success")